In [8]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix


from xgboost import XGBClassifier




In [9]:
def mostrar_metricas(y_test, y_pred):
    print('Reporte de clasificación: \n', metrics.classification_report(y_test, y_pred))
    print('Matriz de confusión: \n', metrics.confusion_matrix (y_test, y_pred))
    print('ROC: \n', metrics.roc_auc_score(y_test, y_pred))

    # Obtener y reformar la matriz de datos de 
    matrix = confusion_matrix (y_test, y_pred) 
    matrix = matrix.astype ('float') / matrix.sum (axis = 1) [:, np.newaxis] 

    fig = px.imshow(matrix, color_continuous_scale='peach', labels=dict(x="Predicción", y="Valor real", color='Porcentaje'),
                x=['Falso', 'Verdadero'],
                y=['Falso', 'Verdadero'])
    fig.update_layout(title_text='Matriz de confusión')
    fig.update(layout=dict(title=dict(x=0.5)))
    fig.show()
def resultados(pred, test_df, algoritmo):
    res_df=pd.DataFrame(test_df['id'])
    res_df['target']=pred
    res_df.to_csv('data/submission' + algoritmo + '.csv', index=False)

ENTRENAMIENTO

In [10]:
train_df = pd.read_csv('data/train_fttd.csv', encoding='utf-8')
test_df = pd.read_csv('data/test_fttd.csv', encoding='utf-8')

X = train_df['text_clean']
y = train_df['target']
t = test_df['text_clean']

#Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7506)

# Doc vs Term
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(X_train)
X_test = count_vect.transform(X_test)

# TF-IDF
tfidf_transformer = TfidfTransformer()
X_train = tfidf_transformer.fit_transform(X_train)
X_test = tfidf_transformer.transform(X_test)

#Entrenamiento
'''
xgb = XGBClassifier(random_state=7506)

params = {"max_depth": [x for x in range(3, 29, 2)],
              "min_child_weight" : [1],
              "n_estimators": [200],
              "learning_rate": [0.1]}
                             
gsearch = GridSearchCV(xgb, param_grid=params, cv = 3, 
                                   verbose=0,n_jobs =-1)

clf = gsearch.fit(X_train, y_train)

print("Best: %f using %s" % (clf.best_score_, clf.best_params_))
'''

'\nxgb = XGBClassifier(random_state=7506)\n\nparams = {"max_depth": [x for x in range(3, 29, 2)],\n              "min_child_weight" : [1],\n              "n_estimators": [200],\n              "learning_rate": [0.1]}\n                             \ngsearch = GridSearchCV(xgb, param_grid=params, cv = 3, \n                                   verbose=0,n_jobs =-1)\n\nclf = gsearch.fit(X_train, y_train)\n\nprint("Best: %f using %s" % (clf.best_score_, clf.best_params_))\n'

In [11]:
#y_pred = clf.predict(X_test)
#mostrar_metricas(y_test, y_pred)

In [12]:
# Doc vs Term
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(X)
X_test = count_vect.transform(t)

# transform  to a normalized tf-idf representation 
tfidf_transformer = TfidfTransformer()
X_train = tfidf_transformer.fit_transform(X_train)
X_test = tfidf_transformer.transform(X_test)


# Train
XGB = XGBClassifier( max_depth=23,  n_estimators=200, random_state=7506)
XGB.fit(X_train, y)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=23,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=7506, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [13]:
y_pred = XGB.predict(X_test)

#Salida
resultados(y_pred, test_df, 'XG&TFIDF')